In [1]:
# P12 - In-class Linear Regress Example
# 스파크의 MLlib을 잘 활용하려면 데이터의 양, feature 추출, 머신러닝 알고리즘 구현, 평가 이 4가지 단계의 파이프 라인이 잘 구성되어야 한다고 느꼈습니다. 그 중 원인 요소들 (feature)을 잘 찾아 결과에 적용될 수 있도록 함수에 공급할 특성(feature)을 찾는 과정 즉, 특성추출이 가장 중요한 단계 같다고 느꼈습니다. 

In [2]:
import findspark

In [3]:
findspark.init('C:\spark-2.4.5-bin-hadoop2.7')

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [6]:
from pyspark.ml.regression import LinearRegression

In [7]:
data = spark.read.csv('Ecommerce_Customers.csv',inferSchema=True, header=True)

In [8]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [9]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [10]:
data.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [11]:
for item in data.head():
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [12]:
from pyspark.ml.linalg import Vectors

In [13]:
from pyspark.ml.feature import VectorAssembler

In [14]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [15]:
assembler = VectorAssembler(inputCols=['Avg Session Length',
                                     'Time on App',
                                     'Time on Website',
                                     'Length of Membership'], outputCol='features')

In [16]:
output = assembler.transform(data)

In [17]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [18]:
output.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [19]:
final_data = output.select('features','Yearly Amount Spent')

In [20]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [21]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                359|
|   mean| 501.49923154603414|
| stddev|  78.71785953849361|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [22]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                141|
|   mean| 493.75031917921393|
| stddev|  80.83051253255417|
|    min|  275.9184206503857|
|    max|  744.2218671047146|
+-------+-------------------+



In [23]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [24]:
lrModel = lr.fit(train_data)

In [25]:
lrModel.coefficients

DenseVector([25.8971, 38.7734, -0.1936, 61.6778])

In [26]:
lrModel.intercept

-1034.9710643789556

In [27]:
test_result = lrModel.evaluate(test_data)

In [28]:
test_result.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|  6.169127324995884|
| -6.861108392571396|
| 10.451633028760398|
|  4.081978501002027|
| 4.3178141827837635|
|-6.5414683999287035|
|-2.5560832703341134|
|-1.6389027161200147|
|-26.060600488765544|
|-1.3561860850849712|
| 1.9353614280385614|
|   8.11523785956632|
| -8.667834348245776|
|   1.46939587357042|
|-1.8411060520711544|
| -10.07717813813008|
| 12.530786259857052|
|-13.317481794648756|
|  8.856675152501452|
| 0.7008647639160586|
+-------------------+
only showing top 20 rows



In [29]:
test_result.rootMeanSquaredError

10.475054852378818

In [30]:
test_result.meanSquaredError

109.72677416034504

In [31]:
test_result.r2

0.9830857391706496

In [32]:
unlabeled_data = test_data.select('features')

In [33]:
predictions = lrModel.transform(unlabeled_data)

In [34]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[31.0472221394875...| 386.3282718640255|
|[31.1280900496166...| 564.1137951396261|
|[31.1695067987115...| 416.9048977735324|
|[31.3584771924370...| 491.0939719484734|
|[31.3662121671876...|426.27106837370115|
|[31.4474464941278...| 425.1442104951527|
|[31.5171218025062...| 278.4745039207198|
|[31.5761319713222...| 542.8654867054483|
|[31.6739155032749...|501.78566839864675|
|[31.7216523605090...| 349.1331127169576|
|[31.7366356860502...|494.99808482749336|
|[31.8209982016720...|416.56004315364703|
|[31.8279790554652...| 448.6705818951873|
|[31.8293464559211...|383.68294211440457|
|[31.8530748017465...|461.12622951442313|
|[31.8648325480987...|449.96845861494376|
|[31.9262720263601...|379.67414718446935|
|[31.9365486184489...|440.51686668997695|
|[31.9549038566348...| 431.1412047874255|
|[32.0047530203648...| 463.0451163567134|
+--------------------+------------